<img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png">

<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

# Walk-forward Optimization

## Install strategy file

Moonshot looks for strategies inside the "moonshot" directory, so execute the following cell to "install" the strategy by moving the file to that location: 

> The `!` sytax below lets us execute terminal commands from inside the notebook. 

In [1]:
# make directory if doesn't exist
!mkdir -p /codeload/moonshot

!mv kitchensink_ml.py /codeload/moonshot/

## Run walk-forward optimization

Now we can run a walk-forward optimization, which iteratively trains and tests the machine learning model over successive periods.

In this example, we retrain the model annually (`train="Y"`) and require 3 years of initial training before any backtesting is performed (`min_train="3Y"`). Additionally, to keep memory usage lower, we use `segment="Y"` to ensure that only a year of data is loaded at a time, even during the 3 year initial training window.   

Because we do not specify a `model` parameter, the default model will be used, which is a regression model using Stochastic Gradient Descent. See the usage guide for more details about the default model.

The parameter `progress=True` causes the moonshot service to log the walk-forward progress during the analysis. To view the progress, open a separate Terminal window and stream the logs (using `cut` to format the logs to fit better in the window): `quantrocket flightlog stream | cut -d ' ' -f 5-`

> As always, it is also a good idea to stream the detailed logs (`quantrocket flightlog stream -d`) to see for more granular logging during the walk-forward optimization.

In [ ]:
from quantrocket.moonshot import ml_walkforward
ml_walkforward("kitchensink-ml",
                start_date="1998-12-31",
                end_date="2018-12-31",
                train="Y",
                progress=True,
                filepath_or_buffer="kitchensink_ml*")

In the terminal window, you'll see output resembling the following:

```
[kitchensink-ml] Walk-forward analysis progress
                train                    test             progress
                start         end       start         end   status Sharpe
iteration
0          1998-12-31  2001-12-30  2001-12-31  2002-12-30        ✓  -0.66
1          2001-12-31  2002-12-30  2002-12-31  2003-12-30        ✓   4.13
2          2002-12-31  2003-12-30  2003-12-31  2004-12-30        ✓   0.59
3          2003-12-31  2004-12-30  2004-12-31  2005-12-30        -
4          2004-12-31  2005-12-30  2005-12-31  2006-12-30
5          2005-12-31  2006-12-30  2006-12-31  2007-12-30
6          2006-12-31  2007-12-30  2007-12-31  2008-12-30
...
```

## Walk-forward results

The walk-forward optimization returns a backtests results CSV and the trained model.

### Backtest results CSV
Open the backtest results CSV like you would for any Moonshot backtest, using Moonchart (and/or pyfolio). 

In [1]:
from moonchart import Tearsheet
Tearsheet.from_moonshot_csv("kitchensink_ml_results.csv")

FileNotFoundError: File b'kitchensink_ml_results.csv' does not exist

### Trained model

The trained model can be opened and inspected with `joblib`:

In [8]:
import joblib
trained_model = joblib.load("kitchensink_ml_trained_model.joblib")
print(type(trained_model))

<class 'sklearn.pipeline.Pipeline'>


In [9]:
estimator = trained_model.named_steps["estimator"]
print(estimator.coef_)

[ 0.04634109 -0.05565511]


To perform live trading, you would need to update your ML strategy to point to this trained model:

```python
    class TheKitchenSinkML(MoonshotML):
        ...
        MODEL = "/codeload/kitchensink_ml/kitchensink_ml_trained_model.joblib"
```

See the usage guide for more details.

***

## *Next Up*

Part 5: [Dimensionality Reduction with PCA](Part5-Dimensionality-Reduction.ipynb)